# Gann Hilo en MACD strategy

Probeer het zo simpel mogelijk te houden...
Initiele uitgangspunten. Exacte instellingen kunnen nog veranderen aan de hand van backtests.

## Indicatoren in gebruik

* Gann Hilo: High period 16, Low period 7
* MACD: Fast 12, Slow 26, Smooth 9

## Idee

* Type: Swingtrade (what's new...)
* Timeframe: 2h

Entry:

1. Als Close > Gann HiLo dan signal buy1
2. Als MACDHIST > 0 dan signal buy2
3. Als buy1 == True AND buy2 == True dan ENTRY TRADE!

Exit:

1. Als Close < Gann HiLo dan EXIT TRADE!

TakeProfit:

* Als 11% target is gehaald, dan take profit

Trailing stoploss:

* Als 8% is gehaald en 2% terug dan take profit

Vaste stoploss:

* bij 25% stoploss (als dit nog van toepassing is...)

In [17]:
# Importeer de benodigde bibliotheken
import pandas as pd
import pandas_ta as pta
from pandas import DataFrame
import numpy as np

In [18]:
# Load data
df = pd.read_json("./data/BTC_USDT-4h.json")
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

,date,open,high,low,close,volume
8433,2021-06-25 12:00:00,33311.96,33600.77,32351.00,32607.07,26868.158019
8434,2021-06-25 16:00:00,32607.07,32910.64,31500.00,32220.74,28395.534991
8435,2021-06-25 20:00:00,32222.91,32369.00,31275.00,31584.45,14423.221134
8436,2021-06-26 00:00:00,31576.09,32268.91,31300.00,31977.59,12066.947988
8437,2021-06-26 04:00:00,31979.71,32730.00,30573.77,30720.37,21059.902932


In [19]:
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.2.45b0
Total Indicators: 133
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_doji, cdl_inside, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, decay, decreasing, dema, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, kama, kc, kdj, kst, kurtosis, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, ssf, stdev, stoch, stochrsi, supertrend, swma, t3, tema, thermo, trend_return, trima, trix, true_range, tsi, ttm_trend, ui, uo, variance, vidya, vortex, vp, vwap, vwma, wcp, willr, wma, zlm

## HILO

In [20]:
help(pta.hilo)

Help on function hilo in module pandas_ta.overlap.hilo:

hilo(high, low, close, high_length=None, low_length=None, mamode=None, offset=None, **kwargs)
    Gann HiLo Activator(HiLo)
    
    The Gann High Low Activator Indicator was created by Robert Krausz in a 1998
    issue of Stocks & Commodities Magazine. It is a moving average based trend
    indicator consisting of two different simple moving averages.
    
    The indicator tracks both curves (of the highs and the lows). The close of the
    bar defines which of the two gets plotted.
    
    Increasing high_length and decreasing low_length better for short trades,
    vice versa for long positions.
    
    Sources:
        https://www.sierrachart.com/index.php?page=doc/StudiesReference.php&ID=447&Name=Gann_HiLo_Activator
        https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/simple-moving-average-sma/
        https://www.tradingview.com/script/XNQSLIYb-Gann-High-Low/
    
    Calculation:
    

In [21]:
# Gann HILO column
pta.hilo(high=df['high'], low=df['low'], close=df['close'], high_length=16, low_length=7, mamode=None, offset=None).tail(5)

,HILO_16_7,HILOl_16_7,HILOs_16_7
8433,34412.608750,NaN,34412.608750
8434,34316.773750,NaN,34316.773750
8435,34190.330000,NaN,34190.330000
8436,34050.886875,NaN,34050.886875
8437,33916.449375,NaN,33916.449375


In [22]:
df['hilo'] = pta.hilo(high=df['high'], low=df['low'], close=df['close'], high_length=16, low_length=7, mamode=None, offset=None)["HILO_16_7"]
df['buy1'] = (df['close'] > df['hilo'])
df[['date','close','hilo','buy1']].tail(10)

,date,close,hilo,buy1
8428,2021-06-24 16:00:00,34857.73,32939.148571,True
8429,2021-06-24 20:00:00,34663.09,33173.578571,True
8430,2021-06-25 00:00:00,35052.07,33388.148571,True
8431,2021-06-25 04:00:00,34283.82,33619.814286,True
8432,2021-06-25 08:00:00,33311.96,34374.248125,False
8433,2021-06-25 12:00:00,32607.07,34412.608750,False
8434,2021-06-25 16:00:00,32220.74,34316.773750,False
8435,2021-06-25 20:00:00,31584.45,34190.330000,False
8436,2021-06-26 00:00:00,31977.59,34050.886875,False
8437,2021-06-26 04:00:00,30720.37,33916.449375,False


## MACD

In [23]:
help(pta.macd)

Help on function macd in module pandas_ta.momentum.macd:

macd(close, fast=None, slow=None, signal=None, offset=None, **kwargs)
    Moving Average Convergence Divergence (MACD)
    
    The MACD is a popular indicator to that is used to identify a security's trend.
    While APO and MACD are the same calculation, MACD also returns two more series
    called Signal and Histogram. The Signal is an EMA of MACD and the Histogram is
    the difference of MACD and Signal.
    
    Sources:
        https://www.tradingview.com/wiki/MACD_(Moving_Average_Convergence/Divergence)
    
    Calculation:
        Default Inputs:
            fast=12, slow=26, signal=9
        EMA = Exponential Moving Average
        MACD = EMA(close, fast) - EMA(close, slow)
        Signal = EMA(MACD, signal)
        Histogram = MACD - Signal
    
    Args:
        close (pd.Series): Series of 'close's
        fast (int): The short period. Default: 12
        slow (int): The long period. Default: 26
        signal (int

In [24]:
pta.macd(close=df['close'], fast=None, slow=None, signal=None, offset=None).tail(5)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
8433,-141.667252,85.050964,-226.718216
8434,-248.586381,-17.494533,-231.091849
8435,-380.280214,-119.350692,-260.929522
8436,-447.763869,-149.467478,-298.296391
8437,-595.824011,-238.022096,-357.801915


In [25]:
df['macdh'] = pta.macd(close=df['close'], fast=None, slow=None, signal=None, offset=None)['MACDh_12_26_9']
df['buy2'] = (df['macdh'] > 0)
df.tail(15)

,date,open,high,low,close,volume,hilo,buy1,macdh,buy2
8423,2021-06-23 20:00:00,32882.50,33785.26,32749.01,33678.07,12525.748531,32868.231429,True,274.265458,True
8424,2021-06-24 00:00:00,33675.07,33879.99,32339.82,32589.20,14734.386878,33509.122500,False,208.923499,True
8425,2021-06-24 04:00:00,32589.20,33280.55,32286.57,32853.74,13051.055908,33512.768125,False,181.154789,True
8426,2021-06-24 08:00:00,32853.74,33930.23,32820.61,33850.77,13506.337429,32873.930000,True,224.724857,True
8427,2021-06-24 12:00:00,33850.78,34583.58,33756.04,33886.52,18730.794267,32889.048571,True,248.865838,True
8428,2021-06-24 16:00:00,33888.81,35298.00,33751.00,34857.73,18670.724202,32939.148571,True,318.695188,True
8429,2021-06-24 20:00:00,34854.63,34999.09,34512.00,34663.09,7932.505576,33173.578571,True,338.372360,True
8430,2021-06-25 00:00:00,34663.08,35500.00,34251.00,35052.07,14634.093456,33388.148571,True,362.057654,True
8431,2021-06-25 04:00:00,35057.70,35100.00,33961.48,34283.82,12494.567347,33619.814286,True,312.336621,True
8432,2021-06-25 08:00:00,34281.37,34480.00,33234.37,33311.96,19245.555409,34374.248125,False,205.506591,True


In [26]:
df['macd'] = pta.macd(close=df['close'], fast=None, slow=None, signal=None, offset=None)['MACD_12_26_9']
df['buy3'] = (df['macd'] > 0)
df.tail(15)

,date,open,high,low,close,volume,hilo,buy1,macdh,buy2,macd,buy3
8423,2021-06-23 20:00:00,32882.50,33785.26,32749.01,33678.07,12525.748531,32868.231429,True,274.265458,True,-573.874848,False
8424,2021-06-24 00:00:00,33675.07,33879.99,32339.82,32589.20,14734.386878,33509.122500,False,208.923499,True,-586.985932,False
8425,2021-06-24 04:00:00,32589.20,33280.55,32286.57,32853.74,13051.055908,33512.768125,False,181.154789,True,-569.465945,False
8426,2021-06-24 08:00:00,32853.74,33930.23,32820.61,33850.77,13506.337429,32873.930000,True,224.724857,True,-469.714662,False
8427,2021-06-24 12:00:00,33850.78,34583.58,33756.04,33886.52,18730.794267,32889.048571,True,248.865838,True,-383.357222,False
8428,2021-06-24 16:00:00,33888.81,35298.00,33751.00,34857.73,18670.724202,32939.148571,True,318.695188,True,-233.854075,False
8429,2021-06-24 20:00:00,34854.63,34999.09,34512.00,34663.09,7932.505576,33173.578571,True,338.372360,True,-129.583813,False
8430,2021-06-25 00:00:00,34663.08,35500.00,34251.00,35052.07,14634.093456,33388.148571,True,362.057654,True,-15.384105,False
8431,2021-06-25 04:00:00,35057.70,35100.00,33961.48,34283.82,12494.567347,33619.814286,True,312.336621,True,12.979017,True
8432,2021-06-25 08:00:00,34281.37,34480.00,33234.37,33311.96,19245.555409,34374.248125,False,205.506591,True,-42.474366,False


## Stochastics als exit signaal?

In [27]:
help(pta.stoch)

Help on function stoch in module pandas_ta.momentum.stoch:

stoch(high, low, close, k=None, d=None, smooth_k=None, offset=None, **kwargs)
    Stochastic (STOCH)
    
    The Stochastic Oscillator (STOCH) was developed by George Lane in the 1950's.
    He believed this indicator was a good way to measure momentum because changes in
    momentum precede changes in price.
    
    It is a range-bound oscillator with two lines moving between 0 and 100.
    The first line (%K) displays the current close in relation to the period's
    high/low range. The second line (%D) is a Simple Moving Average of the %K line.
    The most common choices are a 14 period %K and a 3 period SMA for %D.
    
    Sources:
        https://www.tradingview.com/wiki/Stochastic_(STOCH)
        https://www.sierrachart.com/index.php?page=doc/StudiesReference.php&ID=332&Name=KD_-_Slow
    
    Calculation:
        Default Inputs:
            k=14, d=3, smooth_k=3
        SMA = Simple Moving Average
        LL  = low 

In [28]:
pta.stoch(high=df['high'], low=df['low'], close=df['close'], k=None, d=None, smooth_k=None, offset=None)

,STOCHk_14_3_3,STOCHd_14_3_3
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
8433,36.673696,59.719287
8434,19.967261,39.803896
8435,11.772176,22.804378
8436,13.990703,15.243380


In [32]:
df['stochk']=pta.stoch(high=df['high'], low=df['low'], close=df['close'], k=14, d=4, smooth_k=6, offset=None)['STOCHk_14_4_6']
df['stochd']=pta.stoch(high=df['high'], low=df['low'], close=df['close'], k=14, d=4, smooth_k=6, offset=None)['STOCHd_14_4_6']
df.tail(15)

,date,open,high,low,close,volume,hilo,buy1,macdh,buy2,macd,buy3,stochk,stochd
8423,2021-06-23 20:00:00,32882.50,33785.26,32749.01,33678.07,12525.748531,32868.231429,True,274.265458,True,-573.874848,False,75.926299,69.154213
8424,2021-06-24 00:00:00,33675.07,33879.99,32339.82,32589.20,14734.386878,33509.122500,False,208.923499,True,-586.985932,False,74.574980,72.365701
8425,2021-06-24 04:00:00,32589.20,33280.55,32286.57,32853.74,13051.055908,33512.768125,False,181.154789,True,-569.465945,False,73.256376,73.684281
8426,2021-06-24 08:00:00,32853.74,33930.23,32820.61,33850.77,13506.337429,32873.930000,True,224.724857,True,-469.714662,False,72.722981,74.120159
8427,2021-06-24 12:00:00,33850.78,34583.58,33756.04,33886.52,18730.794267,32889.048571,True,248.865838,True,-383.357222,False,73.810264,73.591150
8428,2021-06-24 16:00:00,33888.81,35298.00,33751.00,34857.73,18670.724202,32939.148571,True,318.695188,True,-233.854075,False,78.169041,74.489666
8429,2021-06-24 20:00:00,34854.63,34999.09,34512.00,34663.09,7932.505576,33173.578571,True,338.372360,True,-129.583813,False,78.591655,75.823485
8430,2021-06-25 00:00:00,34663.08,35500.00,34251.00,35052.07,14634.093456,33388.148571,True,362.057654,True,-15.384105,False,82.922282,78.373311
8431,2021-06-25 04:00:00,35057.70,35100.00,33961.48,34283.82,12494.567347,33619.814286,True,312.336621,True,12.979017,True,83.172757,80.713934
8432,2021-06-25 08:00:00,34281.37,34480.00,33234.37,33311.96,19245.555409,34374.248125,False,205.506591,True,-42.474366,False,74.650297,79.834248
